In [144]:
import numpy as np
import scipy
from scipy.stats import bernoulli
from scipy import stats
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [161]:
elec_2010 = pd.read_csv("Agg_2010.csv")
elec_2012 = pd.read_csv("Agg_2012.csv")
elec_2016 = pd.read_csv("Agg_2016.csv")
elec_2018 = pd.read_csv("Agg_2018.csv")
elec_final = pd.read_csv("Final_Aggregated.csv")
cand_names = pd.read_csv("Primary_Vote_Percentages_Candidates - Sheet3.csv")

In [162]:
elec_final.columns

Index(['Unnamed: 0', 'index', 'Cand_Id', 'Cand_Name', 'Cand_Office_Dist',
       'Cand_Office_St', 'Cand_Party_Affiliation', 'D', 'DISTRICT', 'FEC ID#',
       'Gender', 'Incumbent', 'Median_Income', 'Party_Previous_Vote_Share',
       'Presidential_Approval_Rating', 'Primary.Vote.Percent', 'Raised',
       'STATE', 'Same_Party_As_President', 'Spent', 'Unnamed: 0.1', 'Year',
       'gov_party', 'labor_force', 'minority_percentage', 'name_commonness',
       'percent_bachelor_or_above', 'recent_presidential_vote',
       'seat_transition'],
      dtype='object')

In [163]:
X = elec_final[['Incumbent', 'Median_Income', 'Presidential_Approval_Rating', 'Primary.Vote.Percent', 'Year', 'gov_party', 'labor_force', 'minority_percentage', 'percent_bachelor_or_above', 'recent_presidential_vote']]
X['gov_party'] = (X['gov_party'] == 'DEM').astype('int')

# coding democrats as 1 and republicans as 0
Y = (elec_final['Cand_Party_Affiliation']  == 'DEM').astype('int')

# removing american samoa and guam
X = X.drop(elec_final[elec_final['Cand_Office_St'].isin(['AS', 'GU'])].values[:,0])
Y = Y.drop(elec_final[elec_final['Cand_Office_St'].isin(['AS', 'GU'])].values[:,0])
X.replace('character(0)', 0, inplace = True)
X['labor_force'] = X['labor_force'].astype('float64')
train_indices = X['Year'] != 2016
test_indices = X['Year'] == 2016

D:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [164]:
X = X.values.astype('float64')
Y = Y.values.astype('float64')

In [165]:
# mean imputation on all columns
means = np.nanmean(X, axis = 0) 
for m in range(len(means)):
    X[np.isnan(X[:,m]), m] = means[m]

In [166]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = .2)

In [167]:
X_train = X[train_indices]
X_test = X[test_indices]
y_train = Y[train_indices]
y_test = Y[test_indices]

In [168]:
X_train = X
X_test = elec_2018[['Incumbent', 'Median_Income', 'Presidential_Approval_Rating', 'Primary.Vote.Percent', 'Year', 'gov_party', 'labor_force', 'minority_percentage', 'percent_bachelor_or_above', 'recent_presidential_vote']]
X_test['gov_party'] = (X_test['gov_party'] == 'DEM').astype('int')

X_test.replace('character(0)', 0, inplace = True)
X_test['labor_force'] = X_test['labor_force'].astype('float64')
X_test = X_test.values.astype('float64')
# mean imputation on all columns
means = np.nanmean(X_test, axis = 0) 
for m in range(len(means)):
    X_test[np.isnan(X_test[:,m]), m] = means[m]

y_train = Y


D:\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
D:\anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-ver

In [169]:
clf = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train, y_train)

In [170]:
clf.score(X_train, y_train)

0.6068111455108359

In [171]:
clf.score(X_test, y_test)

ValueError: Found input variables with inconsistent numbers of samples: [419, 435]

In [172]:
clf2 = LogisticRegression(penalty = 'l1').fit(X_train, y_train)
print (clf2.score(X_train, y_train))
print (clf2.score(X_test, y_test))

0.7832817337461301


ValueError: Found input variables with inconsistent numbers of samples: [419, 435]

In [190]:
percent = elec_2018[['Cand_Name', 'Cand_Office_St', 'Cand_Office_Dist','Cand_Party_Affiliation', 'Dem_prob']]

In [198]:
percent[np.logical_and(percent['Cand_Office_St'] == 'AZ',  percent['Cand_Office_Dist'] == 1)].Dem_prob.values[0]

0.18582693118144133

In [177]:
preds = clf2.predict_proba(X_test)
elec_2018['Rep_prob'] = preds[:,0]
elec_2018['Dem_prob'] = preds[:,1]
elec_2018['Cand_Office_Dist'] = elec_2018['Cand_Office_Dist'].astype('int')

In [222]:
df = cand_names.groupby(['State', 'District', 'Party']).apply(max)
name_to_percents = df.groupby(['State', 'District']).apply(lambda x: x.assign(dem_per = percent[np.logical_and(percent['Cand_Office_St'] == x['State'][0],  percent['Cand_Office_Dist'] == x['District'][0])].Dem_prob.values[0]))
only_winners = name_to_percents[np.logical_or(np.logical_and(name_to_percents['dem_per'] > .5, name_to_percents['Party'] == 'DEM'), np.logical_and(name_to_percents['dem_per'] < .5, name_to_percents['Party'] == 'REP'))]
only_winners['win_per'] = np.array([max(x, 1-x) for x in only_winners['dem_per'].values])

D:\anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 'State' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  
D:\anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 'District' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  
D:\anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [226]:
only_winners[['State','District','Name','win_per']].to_csv('preds.csv')

In [119]:
elec_2018

,Unnamed: 0,Unnamed: 0.1,Cand_Name,Cand_Office_St,Cand_Office_Dist,Cand_Party_Affiliation,Year,recent_presidential_vote,percent_bachelor_or_above,gov_party,...,Same_Party_As_President,seat_transition,Median_Income,name_commonness,Primary.Vote.Percent,labor_force,Raised,Spent,Dem_prob,Rep_prob
0,0,10162,"HAFNER, CAROL",AK,0.0,DEM,2018,NaN,28.8,IND,...,False,0.0,73181.0,130,14.8,76.10000,1180622.04,906116.91,0.149356,0.850644
1,1,10164,"ARRAS, JENNIFER O'FARRILL MRS.",AL,1.0,DEM,2018,0.352134,23.3,REP,...,False,0.0,47984.0,1,0.0,74.30000,76091.03,31564.26,0.345353,0.654647
2,2,10169,"AMASON, THOMAS MARRION III",AL,2.0,REP,2018,0.380884,21.8,REP,...,True,4.0,46579.0,20,6.1,76.30000,2479632.21,2065173.18,0.430711,0.569289
3,3,10175,"HAGAN, MALLORY",AL,3.0,DEM,2018,0.404996,21.2,REP,...,False,4.0,46484.0,500,65.8,76.60000,326639.05,302355.16,0.553323,0.446677
4,4,10184,"ADERHOLT, ROBERT B. REP.",AL,4.0,REP,2018,0.215546,16.3,REP,...,True,4.0,43218.0,7,81.5,76.06000,1452763.64,834621.99,0.139949,0.860051
5,5,10187,"BROOKS, MO",AL,5.0,REP,2018,0.326284,30.5,REP,...,True,0.0,54707.0,500,61.3,78.30000,1421378.49,1824411.31,0.390595,0.609405
6,6,10192,"KLINE, DANNER",AL,6.0,DEM,2018,0.424079,35.2,REP,...,False,0.0,65170.0,500,100.0,78.30000,256383.23,161621.12,0.473120,0.526880
7,7,10194,"RICE, RICHARD BRAND NEW",AL,7.0,DEM,2018,0.545119,19.1,REP,...,False,NaN,35988.0,16,0.0,77.04737,1592201.45,868548.99,0.958920,0.041080
8,9,10198,"CRAWFORD, ERIC ALAN RICK",AR,1.0,REP,2018,0.345055,15.7,REP,...,True,4.0,41143.0,14,100.0,78.40000,948321.16,738143.63,0.273503,0.726497
9,10,10205,"COMBS, GWENDOLYNN MILLEN MRS.",AR,2.0,DEM,2018,0.443015,28.4,REP,...,False,4.0,51287.0,500,20.3,77.20000,1670546.59,1440062.32,0.301667,0.698333


In [100]:
X_test[1]

array([0.00000000e+00, 4.79840000e+04, 6.20000000e-01, 0.00000000e+00,
       7.60910300e+04, 3.15642600e+04, 2.01800000e+03, 0.00000000e+00,
       7.43000000e+01, 3.27470000e-01, 1.00000000e+00, 2.33000000e+01,
       3.52134031e-01])

In [124]:
arr = elec_2018[['Cand_Name', 'Cand_Office_St', 'Cand_Office_Dist','Cand_Party_Affiliation', 'Dem_prob']].values

In [137]:
elec_2018[np.logical_or(np.logical_and(arr[:, -1] > .5, arr[:, -2] == 'DEM'), np.logical_and(arr[:, -1] < .5, arr[:, -2] == 'REP'))][['Cand_Name', 'Cand_Office_St', 'Cand_Office_Dist','Cand_Party_Affiliation', 'Dem_prob']]

,Cand_Name,Cand_Office_St,Cand_Office_Dist,Cand_Party_Affiliation,Dem_prob
2,"AMASON, THOMAS MARRION III",AL,2.0,REP,0.430711
3,"HAGAN, MALLORY",AL,3.0,DEM,0.553323
4,"ADERHOLT, ROBERT B. REP.",AL,4.0,REP,0.139949
5,"BROOKS, MO",AL,5.0,REP,0.390595
7,"RICE, RICHARD BRAND NEW",AL,7.0,DEM,0.958920
8,"CRAWFORD, ERIC ALAN RICK",AR,1.0,REP,0.273503
11,"CALDWELL, RANDY",AR,4.0,REP,0.129226
12,"CAVANAUGH, KEVIN",AZ,1.0,REP,0.185342
14,"ABATECOLA, BILL",AZ,3.0,REP,0.111768
19,"ALLEN, CHAD",AZ,8.0,REP,0.033341


In [160]:
cand_names

,"SHEIN, DIMITRI",AK,0,DEM,2018,23.1
0,"YOUNG, DONALD E",AK,0,REP,2018,70.9
1,"KENNEDY, ROBERT JR.",AL,1,DEM,2018,80.8
2,"BYRNE, BRADLEY ROBERTS",AL,1,REP,2018,100.0
3,"ISNER, TABITHA KAY",AL,2,DEM,2018,60.4
4,"BRIGHT, BOBBY NEAL SR.",AL,2,REP,2018,28.1
5,"HAGAN, MALLORY",AL,3,DEM,2018,65.8
6,"ROGERS, MICHAEL",AL,3,REP,2018,100.0
7,"ADERHOLT, ROBERT B. REP.",AL,4,REP,2018,81.5
8,"AUMAN, JAMES LEE",AL,4,DEM,2018,54.1
9,"JOFFRION, PETER S.",AL,5,DEM,2018,100.0


In [159]:
elec_2018['Dem_prob']

0      0.149610
1      0.345068
2      0.430255
3      0.552449
4      0.139656
5      0.389904
6      0.472012
7      0.958795
8      0.273115
9      0.301531
10     0.180576
11     0.129195
12     0.185662
13     0.133580
14     0.112000
15     0.050936
16     0.073995
17     0.053326
18     0.323944
19     0.033426
20     0.192069
21     0.029672
22     0.151199
23     0.238528
24     0.022358
25     0.438295
26     0.600098
27     0.212791
28     0.059591
29     0.284597
         ...   
405    0.141111
406    0.071107
407    0.169754
408    0.817903
409    0.033153
410    0.109034
411    0.745154
412    0.035292
413    0.081479
414    0.067156
415    0.033672
416    0.118742
417    0.044815
418    0.090099
419    0.698203
420    0.087737
421    0.569259
422    0.101660
423    0.021280
424    0.162063
425    0.018731
426    0.551711
427    0.020932
428    0.062937
429    0.017052
430    0.070651
431    0.053745
432    0.221783
433    0.059024
434    0.243399
Name: Dem_prob, Length: 

In [68]:
X_test

,Incumbent,Median_Income,Presidential_Approval_Rating,Primary.Vote.Percent,Raised,Spent,Year,gov_party,labor_force,minority_percentage,name_commonness,percent_bachelor_or_above,recent_presidential_vote
0,0,73181.0,0.48,14.8,1180622.04,906116.91,2018,0,76.10000,0.35812,130,28.8,NaN
1,0,47984.0,0.62,0.0,76091.03,31564.26,2018,0,74.30000,0.32747,1,23.3,0.352134
2,0,46579.0,0.62,6.1,2479632.21,2065173.18,2018,0,76.30000,0.35441,20,21.8,0.380884
3,0,46484.0,0.62,65.8,326639.05,302355.16,2018,0,76.60000,0.31324,500,21.2,0.404996
4,1,43218.0,0.62,81.5,1452763.64,834621.99,2018,0,76.06000,0.13331,7,16.3,0.215546
5,1,54707.0,0.62,61.3,1421378.49,1824411.31,2018,0,78.30000,0.24224,500,30.5,0.326284
6,0,65170.0,0.62,100.0,256383.23,161621.12,2018,0,78.30000,0.21697,500,35.2,0.424079
7,0,35988.0,0.62,0.0,1592201.45,868548.99,2018,0,77.04737,0.66866,16,19.1,0.545119
8,1,41143.0,0.54,100.0,948321.16,738143.63,2018,0,78.40000,0.22750,14,15.7,0.345055
9,0,51287.0,0.54,20.3,1670546.59,1440062.32,2018,0,77.20000,0.29367,500,28.4,0.443015


In [28]:
Y

array([0., 0., 0., ..., 0., 0., 0.])

In [29]:
y_test

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
       0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1.,
       0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 1.

In [31]:
elec_final

,Unnamed: 0,index,Cand_Id,Cand_Name,Cand_Office_Dist,Cand_Office_St,Cand_Party_Affiliation,D,DISTRICT,FEC ID#,...,Spent,Unnamed: 0.1,Year,gov_party,labor_force,minority_percentage,name_commonness,percent_bachelor_or_above,recent_presidential_vote,seat_transition
0,0,3,H6AK00045,"YOUNG, DONALD E",0.0,AK,REP,NaN,0.0,H6AK00045,...,887310.33,3.0,2010.0,REP,78.1,0.32569,42.0,27.0,NaN,4.0
1,1,4,H2AL01077,"BONNER, JO",1.0,AL,REP,NaN,1.0,H2AL01077,...,1101701.00,4.0,2010.0,REP,79.8,0.32626,500.0,20.5,0.390789,4.0
2,2,12,H0AL02087,"ROBY, MARTHA",2.0,AL,REP,NaN,2.0,H0AL02087,...,1240275.64,12.0,2010.0,REP,80,0.35261,161.0,19.3,0.407024,0.0
3,3,14,H2AL03032,"ROGERS, MICHAEL",3.0,AL,REP,NaN,3.0,H2AL03032,...,943059.56,14.0,2010.0,REP,79.34211,0.35445,1.0,19.5,0.456925,4.0
4,4,17,H6AL04098,"ADERHOLT, ROBERT BROWN",4.0,AL,REP,NaN,4.0,H6AL04098,...,969814.03,17.0,2010.0,REP,78.78095,0.08851,7.0,13.1,0.239856,0.0
5,5,18,H0AL05163,"BROOKS, MO",5.0,AL,REP,NaN,5.0,H0AL05163,...,810790.40,18.0,2010.0,REP,77.9,0.23288,500.0,26.7,0.369057,4.0
6,6,25,H2AL06035,"BACHUS, SPENCER T.",6.0,AL,REP,NaN,6.0,H2AL06035,...,1634114.19,25.0,2010.0,REP,78.9,0.17060,500.0,33.6,0.425702,4.0
7,7,36,H0AL07086,"SEWELL, TERRI A.",7.0,AL,DEM,NaN,7.0,H0AL07086,...,1703658.30,36.0,2010.0,REP,78.9,0.65806,148.0,17.1,0.522854,NaN
8,8,44,H0AR01083,"CRAWFORD, ERIC ALAN RICK",1.0,AR,REP,NaN,1.0,H0AR01083,...,1172566.21,44.0,2010.0,DEM,77.2,0.19296,14.0,13.7,0.386129,4.0
9,9,54,H0AR02107,"GRIFFIN, JOHN TIMOTHY",2.0,AR,REP,NaN,2.0,H0AR02107,...,1774143.00,54.0,2010.0,DEM,77,0.25533,6.0,25.8,0.447858,0.0


## Simulation code for determining house control

In [236]:
sum(only_winners['dem_per'].values > .5)

91

In [232]:
dems_take_house_indep(only_winners['dem_per'].values, 100000)

1.0

In [234]:
dems_take_house_corr(only_winners['dem_per'].values, .4, 10000)

0.0705

In [228]:
# probs is an array of probabilities that the democratic candidate will win
def dems_take_house_indep(probs, nsims):
    sims = np.empty((nsims, probs.shape[0]))
    for i in range(probs.shape[0]):
        sims[:,i] = scipy.stats.bernoulli.rvs(probs[i], size = nsims)
    return np.sum(np.sum(sims, axis = 1) > probs.shape[0] / 2) / nsims

In [229]:
# https://twiecki.github.io/blog/2018/05/03/copulas/
def dems_take_house_corr(probs, corr, nsims):
    n = probs.shape[0]
    cov = np.full((n,n), corr)
    np.fill_diagonal(cov, 1)
    mvnorm = scipy.stats.multivariate_normal(mean=np.zeros(n), cov = cov)
    x = mvnorm.rvs(nsims)
    x_unif = stats.norm.cdf(x)
    sims = np.empty((nsims, probs.shape[0]))
    for i in range(probs.shape[0]):
        sims[:,i] = bernoulli.ppf(x_unif[:,i], probs[i])
    return np.sum(np.sum(sims, axis = 1) > probs.shape[0] / 2) / nsims


In [108]:
elec_2010 = pd.read_csv("Agg_2010-1.csv")
elec_2012 = pd.read_csv("Agg_2012.csv")
elec_2014 = pd.read_csv("Agg_2014-1.csv")
elec_2016 = pd.read_csv("Agg_2016-1.csv")

In [117]:
elec_2014[30:60]

,Unnamed: 0,Unnamed: 0.1,Cand_Name,Cand_Office_St,Cand_Office_Dist,Cand_Party_Affiliation,Year,recent_presidential_vote,percent_bachelor_or_above,gov_party,Party_Previous_Vote_Share,Incumbent,minority_percentage,Gender,Presidential_Approval_Rating,Same_Party_As_President,seat_transition,Median_Income,name_commonness,Primary.Vote.Percent,labor_force,Raised,Spent,STATE,D,FEC ID#,Cand_Id,GE WINNER INDICATOR
30,30,30,"ROBINSON, JOHNNIE ELBERT",AZ,7.0,DEM,2014,0.457491,13.6,REP,0.817425,0,0.37018,1.0,0.40,True,4.0,34881.0,500,NaN,77.34286,936188.95,911026.32,NaN,NaN,NaN,NaN,NaN
31,31,31,"ROGERS, TED",AZ,7.0,LIB,2014,0.457491,13.6,REP,0.182575,0,0.37018,1.0,0.40,False,4.0,34881.0,181,22.44,77.34286,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,32,32,"WASSERMAN, BRIANNA",AZ,7.0,REP,2014,0.457491,13.6,REP,0.000000,0,0.37018,0.0,0.40,False,4.0,34881.0,500,NaN,77.34286,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,33,33,"PARKER, VERNON",AZ,9.0,REP,2014,NaN,35.2,REP,0.446286,0,0.23608,1.0,0.40,False,NaN,51106.0,500,NaN,74.8,1378441.10,1328058.64,NaN,NaN,NaN,NaN,NaN
34,34,34,"SEPULVEDA, MARTIN C.",AZ,9.0,REP,2014,NaN,35.2,REP,0.446286,0,0.23608,1.0,0.40,False,NaN,51106.0,115,NaN,74.8,1378441.10,1328058.64,NaN,NaN,NaN,NaN,NaN
35,35,35,"VICTOR, MARC J",AZ,9.0,REP,2014,NaN,35.2,REP,0.446286,0,0.23608,1.0,0.40,False,NaN,51106.0,89,NaN,74.8,1378441.10,1328058.64,NaN,NaN,NaN,NaN,NaN
36,36,36,"HARTER, RYAN",CA,3.0,IND,2014,0.587945,23.7,DEM,0.000000,0,0.34647,1.0,0.51,False,0.0,56275.0,25,NaN,74.64211,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,37,37,"DACE, CHRISTOPHER MICHAEL",CA,4.0,REP,2014,0.507321,31.4,DEM,0.611092,0,0.13787,1.0,0.51,False,3.0,68539.0,2,NaN,73.05714,231830.17,196522.01,NaN,NaN,NaN,NaN,NaN
38,38,38,"JOHNSON, KRISTINE M",CA,4.0,DEM,2014,0.507321,31.4,DEM,0.000000,0,0.13787,0.0,0.51,True,3.0,68539.0,143,NaN,73.05714,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,39,39,"WEBB, JOHN",CA,4.0,REP,2014,0.507321,31.4,DEM,0.611092,0,0.13787,1.0,0.51,False,3.0,68539.0,6,NaN,73.05714,231830.17,196522.01,NaN,NaN,NaN,NaN,NaN


In [109]:
print (np.sum(elec_2010['GE WINNER INDICATOR'] == 'W'))
print (np.sum(elec_2012['GE WINNER INDICATOR'] == 'W'))
print (np.sum(elec_2014['GE WINNER INDICATOR'] == 'W'))
print (np.sum(elec_2016['GE WINNER INDICATOR'] == 'W'))


437
438
437
440


In [83]:
elec_2010['Primary.Vote.Percent'].fillna(0, inplace = True)

In [129]:
group = elec_2010.groupby(['Cand_Office_St', 'Cand_Office_Dist', 'Cand_Party_Affiliation'])['Primary.Vote.Percent'].transform(max) == elec_2010['Primary.Vote.Percent']

KeyError: "None of [['Incumbent', 'Median_Income', 'Presidential_Approval_Rating', 'Primary.Vote.Percent', 'Raised', 'Same_Party_As_President', 'Spent', 'Year', 'gov_party', 'labor_force', 'minority_percentage', 'name_commonness', 'percent_bachelor_or_above', 'recent_presidential_vote']] are in the [index]"

In [79]:
pd.set_option('display.max_columns', None)

In [86]:
elec_2010[group]

,Unnamed: 0,Cand_Name,Cand_Office_St,Cand_Office_Dist,Cand_Party_Affiliation,Year,recent_presidential_vote,percent_bachelor_or_above,gov_party,Party_Previous_Vote_Share,Incumbent,minority_percentage,Gender,Presidential_Approval_Rating,Same_Party_As_President,seat_transition,Median_Income,name_commonness,Primary.Vote.Percent,labor_force,Raised,Spent,STATE_x,D_x,FEC ID#,Cand_Id,STATE_y,D_y,GENERAL,GENERAL %,"COMBINED GE PARTY TOTALS (CT, NY, SC)","COMBINED % (CT, NY, SC)",total,GE WINNER INDICATOR
1,1,"CRAWFORD, HARRY T JR",AK,0.0,DEM,2010,NaN,27.0,REP,0.451464,0,0.32569,1.0,0.38,True,4.0,64576.0,200,100.00,78.1,240439.24,235571.43,Alaska,0.0,H0AK00089,H0AK00089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"YOUNG, DONALD E",AK,0.0,REP,2010,NaN,27.0,REP,0.503333,1,0.32569,1.0,0.38,False,4.0,64576.0,42,70.35,78.1,1001015.37,887310.33,Alaska,0.0,H6AK00045,H6AK00045,Alaska,0.0,175384.0,0.689579,NaN,NaN,0.689579,W
4,4,"BONNER, JO",AL,1.0,REP,2010,0.390789,20.5,REP,1.000000,1,0.32626,1.0,0.41,False,4.0,41172.0,500,75.25,79.8,913052.62,1101701.00,Alabama,1.0,H2AL01077,H2AL01077,Alabama,1.0,129063.0,0.825839,NaN,NaN,0.825839,W
6,6,"WALTER, DAVID MARSH",AL,1.0,CON,2010,0.390789,20.5,REP,0.000000,0,0.32626,1.0,0.41,False,4.0,41172.0,4,0.00,79.8,NaN,NaN,Alabama,1.0,H0AL01048,H0AL01048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,"BRIGHT SR, BOBBY NEAL",AL,2.0,DEM,2010,0.407024,19.3,REP,0.503119,0,0.35261,1.0,0.41,True,0.0,40567.0,500,0.00,80,1413031.76,1435526.44,Alabama,2.0,H8AL02130,H8AL02130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,"JOHN, ROBERT E JR",AL,2.0,IND,2010,0.407024,19.3,REP,0.000000,0,0.35261,1.0,0.41,False,0.0,40567.0,7,0.00,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,11,"LAYFIELD, MARK",AL,3.0,IND,2010,0.456925,19.5,REP,0.000000,0,0.35445,1.0,0.41,False,4.0,35452.0,18,0.00,79.34211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,20,"LANCASTER, ROSIE FUALA'AU TAGO",AS,0.0,IND,2010,NaN,NaN,DEM,0.000000,0,NaN,0.0,NaN,False,NaN,NaN,500,0.00,character(0),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,26,"SCOTT, BRIAN ALAN",AZ,3.0,IND,2010,0.445441,33.9,REP,0.000000,0,0.14390,1.0,0.43,False,1.0,52327.0,8,0.00,76.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,27,"HILL, BRIAN",AZ,4.0,IND,2010,0.445441,13.6,REP,0.000000,0,0.31865,1.0,0.43,False,0.0,31498.0,8,0.00,77.19474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
elec_2012['Primary.Vote.Percent'].fillna(0, inplace = True)
elec_2012['Primary.Vote.Percent'].replace('#VALUE!', 0, inplace = True)
elec_2012['Primary.Vote.Percent'] = elec_2012['Primary.Vote.Percent'].astype('float')

In [160]:
elec_2012group = elec_2012.groupby(['Cand_Office_St', 'Cand_Office_Dist', 'Cand_Party_Affiliation'])['Primary.Vote.Percent'].transform(max) == elec_2012['Primary.Vote.Percent']

In [ ]:
elec_2014['Primary.Vote.Percent'].fillna(0, inplace = True)
elec_2014['Primary.Vote.Percent'].replace('#VALUE!', 0, inplace = True)
elec_2014['Primary.Vote.Percent'] = elec_2014['Primary.Vote.Percent'].astype('float')

In [ ]:
elec_2014group = elec_2014.groupby(['Cand_Office_St', 'Cand_Office_Dist', 'Cand_Party_Affiliation'])['Primary.Vote.Percent'].transform(max) == elec_2014['Primary.Vote.Percent']

In [ ]:
elec_2016['Primary.Vote.Percent'].fillna(0, inplace = True)
elec_2016['Primary.Vote.Percent'].replace('#VALUE!', 0, inplace = True)
elec_2016['Primary.Vote.Percent'] = elec_2016['Primary.Vote.Percent'].astype('float')

In [ ]:
elec_2016group = elec_2016.groupby(['Cand_Office_St', 'Cand_Office_Dist', 'Cand_Party_Affiliation'])['Primary.Vote.Percent'].transform(max) == elec_2016['Primary.Vote.Percent']

In [202]:
monstrosity = elec_2012[group2012][elec_2012[group2012]['Cand_Party_Affiliation'].isin(['DEM', 'REP'])].groupby(['Cand_Office_St', 'Cand_Office_Dist'])[['Cand_Office_St', 'Cand_Party_Affiliation','Incumbent', 'Median_Income', 'Presidential_Approval_Rating', 'Primary.Vote.Percent', 'Raised', 'Same_Party_As_President', 'Spent', 'Year', 'gov_party', 'labor_force', 'minority_percentage', 'name_commonness', 'percent_bachelor_or_above', 'recent_presidential_vote']].apply(lambda x: x)

In [59]:
elec_2010.columns

Index(['Unnamed: 0', 'Cand_Name', 'Cand_Office_St', 'Cand_Office_Dist',
       'Cand_Party_Affiliation', 'Year', 'recent_presidential_vote',
       'percent_bachelor_or_above', 'gov_party', 'Party_Previous_Vote_Share',
       'Incumbent', 'minority_percentage', 'Gender',
       'Presidential_Approval_Rating', 'Same_Party_As_President',
       'seat_transition', 'Median_Income', 'name_commonness',
       'Primary.Vote.Percent', 'labor_force', 'Raised', 'Spent', 'STATE_x',
       'D_x', 'FEC ID#', 'Cand_Id', 'STATE_y', 'D_y', 'GENERAL ', 'GENERAL %',
       'COMBINED GE PARTY TOTALS (CT, NY, SC)', 'COMBINED % (CT, NY, SC)',
       'total', 'GE WINNER INDICATOR'],
      dtype='object')

In [60]:
elec_2012.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Cand_Name', 'Cand_Office_St',
       'Cand_Office_Dist', 'Cand_Party_Affiliation', 'Year',
       'recent_presidential_vote', 'percent_bachelor_or_above', 'gov_party',
       'Party_Previous_Vote_Share', 'Incumbent', 'minority_percentage',
       'Gender', 'Presidential_Approval_Rating', 'Same_Party_As_President',
       'seat_transition', 'Median_Income', 'name_commonness',
       'Primary.Vote.Percent', 'labor_force', 'Raised', 'Spent', 'STATE', 'D',
       'FEC ID#', 'Cand_Id', 'GE RUNOFF ELECTION VOTES (LA)',
       'GE RUNOFF ELECTION % (LA)', 'COMBINED GE PARTY TOTALS (CT, NY, SC)',
       'COMBINED % (CT, NY, SC)', 'GE WINNER INDICATOR', 'Combined totals',
       'Combined %'],
      dtype='object')

In [50]:
elec_2010.head()

,Unnamed: 0,Cand_Name,Cand_Office_St,Cand_Office_Dist,Cand_Party_Affiliation,Year,recent_presidential_vote,percent_bachelor_or_above,gov_party,Party_Previous_Vote_Share,...,FEC ID#,Cand_Id,STATE_y,D_y,GENERAL,GENERAL %,"COMBINED GE PARTY TOTALS (CT, NY, SC)","COMBINED % (CT, NY, SC)",total,GE WINNER INDICATOR
0,0,"COX, JOHN R.",AK,0.0,REP,2010,NaN,27.0,REP,0.503333,...,H0AK00097,H0AK00097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"CRAWFORD, HARRY T JR",AK,0.0,DEM,2010,NaN,27.0,REP,0.451464,...,H0AK00089,H0AK00089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"FISHER, SHELDON ALLRED",AK,0.0,REP,2010,NaN,27.0,REP,0.503333,...,H0AK01038,H0AK01038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"YOUNG, DONALD E",AK,0.0,REP,2010,NaN,27.0,REP,0.503333,...,H6AK00045,H6AK00045,Alaska,0.0,175384.0,0.689579,NaN,NaN,0.689579,W
4,4,"BONNER, JO",AL,1.0,REP,2010,0.390789,20.5,REP,1.000000,...,H2AL01077,H2AL01077,Alabama,1.0,129063.0,0.825839,NaN,NaN,0.825839,W
